# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""14701768515961094324""",4.3868e18,"""../../../../data/SampleHDS.jso…","""6933706528673508779""","""8329648895030493763""","""4912967725802745490""",false
"""1830870434762229531""",1.6294e19,"""../../../../data/SampleHDS.jso…","""15048428407909383056""","""8329648895030493763""","""13285166811542217445""",true
"""10703286009199392865""",6.1954e18,"""../../../../data/SampleHDS.jso…","""374362652987472736""","""7232539414939600107""","""4912967725802745490""",false
"""2306871788416096254""",9.9240e18,"""../../../../data/SampleHDS.jso…","""15048428407909383056""","""2641638384544890442""","""4912967725802745490""",false
"""1563471117808725661""",1.7078e19,"""../../../../data/SampleHDS.jso…","""6933706528673508779""","""2641638384544890442""","""12072159323336673559""",true
"""10209740610312381670""",5.0893e17,"""../../../../data/SampleHDS.jso…","""374362652987472736""","""5716144096203697984""",null,false
"""5320432496449302954""",3.0313e18,"""../../../../data/SampleHDS.jso…","""374362652987472736""","""2641638384544890442""","""12072159323336673559""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'Customer_CLV': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""5448452337218064398""",2.8499e17,"""../../../../data/SampleHDS.jso…","""15828018352702081594""","""FirstMortgage30yr""","""12869474465175321465""","""Rejected"""
"""17478127141569231282""",1.6206e19,"""../../../../data/SampleHDS.jso…","""16354800262861975004""","""FirstMortgage30yr""","""2826875804395316157""","""Accepted"""
"""17002059151292482706""",2.5110e18,"""../../../../data/SampleHDS.jso…","""1602248905436116730""","""MoneyMarketSavingsAccount""","""12869474465175321465""","""Rejected"""
"""13130913155690388550""",4.0151e18,"""../../../../data/SampleHDS.jso…","""16354800262861975004""","""BasicChecking""","""12869474465175321465""","""Rejected"""
"""13083853053692937498""",3.3418e18,"""../../../../data/SampleHDS.jso…","""15828018352702081594""","""BasicChecking""","""7383630733164936082""","""Accepted"""
"""17836924343033675323""",1.0085e19,"""../../../../data/SampleHDS.jso…","""1602248905436116730""","""UPlusFinPersonal""",null,"""Rejected"""
"""15528079268958571530""",1.5180e19,"""../../../../data/SampleHDS.jso…","""1602248905436116730""","""BasicChecking""","""7383630733164936082""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""14358030279554259035""",1.7053e19,"""../../../../data/SampleHDS.jso…","""3379665622621738913""","""FirstMortgage30yr""","""11730390344365785675""","""Rejected"""
"""9786188987779878491""",8.4606e18,"""../../../../data/SampleHDS.jso…","""12225859183858212282""","""FirstMortgage30yr""","""13611860230996235604""","""Accepted"""
"""11776150188831750100""",1.4298e19,"""../../../../data/SampleHDS.jso…","""4400466202206793792""","""MoneyMarketSavingsAccount""","""11730390344365785675""","""Rejected"""
"""3371989351220163991""",1.8222e19,"""../../../../data/SampleHDS.jso…","""12225859183858212282""","""BasicChecking""","""11730390344365785675""","""Rejected"""
"""5441665109427285436""",2.7539e18,"""../../../../data/SampleHDS.jso…","""3379665622621738913""","""BasicChecking""","""13158039235640228738""","""Accepted"""
"""13102695026656957626""",1.4071e19,"""../../../../data/SampleHDS.jso…","""4400466202206793792""","""UPlusFinPersonal""",null,"""Rejected"""
"""6029228625241794598""",1.0575e19,"""../../../../data/SampleHDS.jso…","""4400466202206793792""","""BasicChecking""","""13158039235640228738""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
Customer_CLV=PREDICTOR_1
filename=filename
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""8262429355211842360""",8.3867e18,"""12284549990905224385""","""FirstMortgage30yr""","""11589024074427623291""","""Rejected"""
"""7966456522498371456""",8.9327e18,"""9665925817762485216""","""FirstMortgage30yr""","""6844352117297209851""","""Accepted"""
"""4855127628707324881""",6.1854e18,"""8320858726954197783""","""MoneyMarketSavingsAccount""","""11589024074427623291""","""Rejected"""
"""2935096018552473823""",9.2904e18,"""9665925817762485216""","""BasicChecking""","""11589024074427623291""","""Rejected"""
"""15295666770933943866""",2.2013e17,"""12284549990905224385""","""BasicChecking""","""3758632175384387202""","""Accepted"""
"""3613709968155097108""",2.4484e18,"""8320858726954197783""","""UPlusFinPersonal""",null,"""Rejected"""
"""9397904830987699640""",1.0939e19,"""8320858726954197783""","""BasicChecking""","""3758632175384387202""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'